In [1]:
import numpy as np
import pandas as pd
pd.options.display.precision = 2
pd.options.display.max_columns = 50

In [2]:
import re
import os
os.chdir('../../')

In [3]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [15, 10]

In [4]:
#import seaborn as sns
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Read Predictors data

In [5]:
!pwd

/Users/shijunyi/Desktop/������������/U of Melbourne/2019 Sem2/Data Science Project


In [6]:
predictors_df = pd.read_csv('data/Primary_Commodity_Price_System_PCPS.csv', skiprows=1)

FileNotFoundError: File b'data/Primary_Commodity_Price_System_PCPS.csv' does not exist

In [7]:
predictors_df['year'] = predictors_df.Date.apply(lambda x: int(x.split(' ')[1]))
#predictors_df.head()

In [8]:
predictors_yearly_df = predictors_df.groupby('year').mean()
cols = predictors_yearly_df.columns
predictors_yearly_df = predictors_yearly_df.join(predictors_yearly_df[cols].shift(), rsuffix='_1y_shift')
predictors_yearly_df = predictors_yearly_df.join(predictors_yearly_df[cols].shift(2), rsuffix='_2y_shift')
predictors_yearly_df = predictors_yearly_df.loc[2003:2018]

In [9]:
predictors_yearly_df.head()

,Index,Energy index,Non-Fuel index,Coal index,Natural gas index,APSP crude oil_USD_per_bbl,Propane,Industrial Materials index,Food and beverage index,Fertilizer,Index_1y_shift,Energy index_1y_shift,Non-Fuel index_1y_shift,Coal index_1y_shift,Natural gas index_1y_shift,APSP crude oil_USD_per_bbl_1y_shift,Propane_1y_shift,Industrial Materials index_1y_shift,Food and beverage index_1y_shift,Fertilizer_1y_shift,Index_2y_shift,Energy index_2y_shift,Non-Fuel index_2y_shift,Coal index_2y_shift,Natural gas index_2y_shift,APSP crude oil_USD_per_bbl_2y_shift,Propane_2y_shift,Industrial Materials index_2y_shift,Food and beverage index_2y_shift,Fertilizer_2y_shift
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003,65.99,78.15,57.58,43.29,122.67,66.92,118.85,56.87,67.88,63.89,56.14,62.99,51.41,39.49,85.20,62.37,84.21,48.89,62.27,49.79,56.89,66.68,50.12,49.34,103.53,61.95,97.21,48.65,60.23,54.60
2004,79.37,98.81,65.94,82.92,134.16,82.50,152.77,72.09,73.98,74.88,65.99,78.15,57.58,43.29,122.67,66.92,118.85,56.87,67.88,63.89,56.14,62.99,51.41,39.49,85.20,62.37,84.21,48.89,62.27,49.79
2005,97.94,136.64,71.19,71.92,195.76,117.35,187.74,83.98,75.95,92.23,79.37,98.81,65.94,82.92,134.16,82.50,152.77,72.09,73.98,74.88,65.99,78.15,57.58,43.29,122.67,66.92,118.85,56.87,67.88,63.89
2006,114.36,154.33,86.73,76.88,189.10,141.65,208.49,121.26,81.92,89.30,97.94,136.64,71.19,71.92,195.76,117.35,187.74,83.98,75.95,92.23,79.37,98.81,65.94,82.92,134.16,82.50,152.77,72.09,73.98,74.88
2007,129.29,169.42,101.54,98.37,193.41,150.35,248.43,144.61,93.37,129.20,114.36,154.33,86.73,76.88,189.10,141.65,208.49,121.26,81.92,89.30,97.94,136.64,71.19,71.92,195.76,117.35,187.74,83.98,75.95,92.23


# Read response data (Sponsored Students Counts)

In [11]:
response_df = pd.read_csv("data/SponsorCompany_ year_backup.csv").set_index('Company')
response_df.columns.name = 'year'
response_df = response_df.stack()
response_df.name = 'students_count'
response_df = response_df.to_frame().reset_index()
response_df['year'] = response_df.year.astype(int)
response_df.to_csv('data/sponsored_students_counts_byCompany.csv', index=False)

In [12]:
mask = response_df.Company.apply(lambda x: re.search('.*Total.*', x) is None)
response_df = response_df[mask]

In [13]:
years_df = response_df.groupby('year').students_count.sum().sort_index()

traces = []
traces.append(go.Bar(
        x=years_df.index.values,
        y=years_df.values,
        name='Sponsored Student Count',
        opacity = 0.5))


layout = dict(
    title = 'Total Number of Sponsored Students By Year',
    xaxis = dict(title = 'Year'),
    
    yaxis = dict(
        title = 'Number of Sponsored Students',
        #range = [0, 250]
    ),
    
    legend=dict(
        x=0.3,
        y=1.1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='#000'
        ),
        orientation="h"
    )
,
)

fig = dict(data=traces, layout=layout)
iplot(fig)

In [14]:
top_companies_df = response_df.groupby('Company').students_count.sum().sort_values(ascending=False).head(100)
top_companies = top_companies_df.index.values

In [15]:
traces = []
traces.append(go.Bar(
        x=top_companies_df.index.values,
        y=top_companies_df.values,
        name='Sponsored Student Count',
        opacity = 0.5))


layout = dict(
    title = 'Total Number of Sponsored Students By Top 100 Companies',
    xaxis = dict(title = 'Year'),
    
    yaxis = dict(
        title = 'Number of Sponsored Students',
        #range = [0, 250]
    ),
    
    legend=dict(
        x=0.3,
        y=1.1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='#000'
        ),
        orientation="h"
    )
,
)

fig = dict(data=traces, layout=layout)
iplot(fig)

In [16]:
training_set = response_df.join(predictors_yearly_df, on='year')
training_set.to_csv('data/training_set_companies.csv')

In [18]:
training_set.set_index(['Company', 'year'], inplace=True)

# Visualise

## Correlation

In [21]:
df = response_df.set_index(['Company', 'year']).unstack('Company')
df.columns = df.columns.droplevel(0)
df = df.loc[:2018][top_companies]

In [22]:
correlation_df = predictors_yearly_df.join(df).corr()
correlation_df = correlation_df[predictors_yearly_df.columns].loc[list(df.columns.values)]
correlation_df

,Index,Energy index,Non-Fuel index,Coal index,Natural gas index,APSP crude oil_USD_per_bbl,Propane,Industrial Materials index,Food and beverage index,Fertilizer,Index_1y_shift,Energy index_1y_shift,Non-Fuel index_1y_shift,Coal index_1y_shift,Natural gas index_1y_shift,APSP crude oil_USD_per_bbl_1y_shift,Propane_1y_shift,Industrial Materials index_1y_shift,Food and beverage index_1y_shift,Fertilizer_1y_shift,Index_2y_shift,Energy index_2y_shift,Non-Fuel index_2y_shift,Coal index_2y_shift,Natural gas index_2y_shift,APSP crude oil_USD_per_bbl_2y_shift,Propane_2y_shift,Industrial Materials index_2y_shift,Food and beverage index_2y_shift,Fertilizer_2y_shift
Australia Awards,0.69,0.59,7.64e-01,3.94e-01,0.25,0.68,1.56e-01,6.20e-01,0.79,0.51,8.66e-01,8.12e-01,8.76e-01,7.41e-01,0.53,8.51e-01,0.55,0.79,8.72e-01,7.70e-01,0.85,0.80,8.68e-01,0.88,0.59,8.03e-01,0.70,0.88,8.06e-01,8.47e-01
Public Service Depart.Malaysia,0.31,0.27,3.30e-01,1.93e-01,0.17,0.27,2.66e-01,4.05e-01,0.27,0.34,3.67e-01,4.18e-01,2.65e-01,3.16e-01,0.47,3.67e-01,0.36,0.27,2.60e-01,5.65e-01,0.54,0.63,3.80e-01,0.61,0.80,5.48e-01,0.71,0.43,3.73e-01,6.94e-01
MARA-Educ.Sponsorship Division,0.49,0.45,4.97e-01,3.02e-01,0.32,0.46,2.72e-01,4.70e-01,0.47,0.59,7.52e-01,8.06e-01,6.16e-01,7.45e-01,0.80,7.49e-01,0.74,0.62,6.16e-01,8.76e-01,0.62,0.65,5.46e-01,0.56,0.68,6.05e-01,0.75,0.68,4.70e-01,5.98e-01
Saudi Arabian Cultural Mission,0.66,0.50,8.33e-01,5.30e-01,0.07,0.60,2.09e-01,7.16e-01,0.81,0.45,7.12e-01,6.08e-01,8.05e-01,7.15e-01,0.30,6.48e-01,0.41,0.73,7.83e-01,6.68e-01,0.74,0.68,7.71e-01,0.78,0.51,6.79e-01,0.55,0.73,7.50e-01,8.43e-01
CONICYT-Chile,0.59,0.48,7.03e-01,3.15e-01,0.12,0.59,2.02e-02,5.26e-01,0.74,0.41,8.47e-01,7.88e-01,8.66e-01,7.62e-01,0.53,8.29e-01,0.45,0.73,8.80e-01,8.39e-01,0.87,0.81,9.10e-01,0.86,0.58,8.18e-01,0.73,0.92,8.58e-01,8.22e-01
MU discount for SENESCYT,0.36,0.36,3.23e-01,-3.57e-02,0.28,0.43,-3.55e-02,1.67e-01,0.45,0.14,5.60e-01,5.64e-01,5.11e-01,2.47e-01,0.40,6.44e-01,0.15,0.39,5.50e-01,3.61e-01,0.64,0.60,6.57e-01,0.55,0.37,6.45e-01,0.42,0.62,6.17e-01,5.08e-01
Endeavour Programme,0.58,0.47,6.82e-01,4.64e-01,0.16,0.53,2.08e-01,5.44e-01,0.72,0.41,6.06e-01,5.23e-01,6.77e-01,5.40e-01,0.25,5.85e-01,0.34,0.64,6.73e-01,4.09e-01,0.59,0.54,6.31e-01,0.54,0.34,5.77e-01,0.30,0.54,6.23e-01,6.27e-01
Indonesia Endowment Fund for Education,-0.21,-0.36,3.27e-02,7.03e-03,-0.57,-0.25,-5.77e-01,-2.00e-01,0.13,-0.26,-1.08e-03,-1.30e-01,1.82e-01,-8.33e-03,-0.27,-3.44e-02,-0.44,-0.04,2.90e-01,-4.14e-02,0.33,0.26,4.08e-01,0.12,0.11,3.49e-01,-0.11,0.21,5.26e-01,1.69e-01
Ministry of Education Malaysia,0.30,0.33,2.28e-01,3.47e-01,0.40,0.24,3.84e-01,2.68e-01,0.19,0.67,4.05e-01,4.90e-01,2.52e-01,4.11e-01,0.65,3.94e-01,0.67,0.39,2.09e-01,5.51e-01,0.20,0.27,8.43e-02,0.06,0.46,2.22e-01,0.50,0.29,2.07e-04,8.86e-02
Conselho Nacion Desenv Cientif e Tecnolog/Latino,0.34,0.33,3.35e-01,-2.79e-02,0.23,0.41,-1.20e-01,1.73e-01,0.45,0.15,5.95e-01,5.90e-01,5.56e-01,2.84e-01,0.42,6.68e-01,0.18,0.43,5.91e-01,4.25e-01,0.68,0.63,7.01e-01,0.60,0.39,6.70e-01,0.50,0.69,6.61e-01,4.92e-01


## Heatmap

In [24]:
df = correlation_df.loc[correlation_df.index.values[::-1]]
trace = go.Heatmap(z=df.values,
                   x=df.columns.values,
                   y=df.index.values)

layout = go.Layout(
    title="Correlation Between the Number of Sponsored Students and Commodity Indices",
    autosize=False,
    width=1000,
    height=1000)

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

## Comparison Chart

In [25]:
predictors = ['Index',
              'Energy index',
              'Non-Fuel index',
              #'Coal index', 'Natural gas index',
       #'APSP crude oil_USD_per_bbl', 'Propane', 'Industrial Materials index',
       #'Food and beverage index', 'Fertilizer'
             ]

In [45]:
traces = []

# Adding predictor lines
for predictor in predictors:
    traces.append(
        go.Scatter(
            x=predictors_yearly_df.index.values,
            y=predictors_yearly_df[predictor].values,
            name=predictor,
            mode='lines'
        ),
    )

# Adding response bar-chart by country
#countries = np.sort(response_df.country.unique())
companies = top_companies
visibility = True
for Company in companies:
    df = response_df[response_df.Company == Company]
    x = df.year.values
    traces.append(go.Bar(
        x=x,
        y=df.students_count,
        yaxis='y2',
        visible=visibility,
        name='Sponsored Student Count',
        opacity = 0.5))
    visibility = False


# Adding dropdown menu    
buttons = []
pr_count = len(predictors)
for idx, Company in enumerate(companies):
    mask = [True] * pr_count + [False] * len(companies)
    mask[pr_count + idx] = True
    buttons.append(
        dict(
            label = Company,
            method = 'update',
            args = [{'visible': mask}]
        )
    )
    
    
layout = dict(
    title = 'Trends in Commodity Prices and Number of Sponsored Students by Company',
    
    xaxis = dict(title = 'Year'),
    
    yaxis = dict(
        title = 'Commodity Index Value',
        range = [0, 350]
    ),
    
    yaxis2 = dict(
        title='Sponsored Student Count',
        overlaying='y',
        side='right',
        range=[0,1000]
        
    ),
    updatemenus = list([dict(
            active=0,
            pad = {'r': 0, 't': 1},
            x = 0.0,
            xanchor = 'left',
            y = 1.1,
            yanchor = 'top',
            buttons=buttons)]),
    legend=dict(
        x=0.9,
        y=1.0,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='#000'
        ),
        orientation="h"
    )
,
)


fig = dict(data=traces, layout=layout)

iplot(fig)